In [1]:
from ckiptagger import construct_dictionary, WS, POS, NER
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import *
from summa import keywords
import string
import re

In [2]:
model_path = './TokSentLeo/CKIP_model/'
data_dir = "D:/python/LEO_TM/BERT_chinese_LM_processing/Data/keyword/"
ws = WS(model_path)
pos = POS(model_path)
ner = NER(model_path)

In [3]:
df_ori = pd.read_excel(data_dir+"Item_10F.xlsx")
df = df_ori
df_ori

,品名
0,K1_RF無機SCRUBBER_A105水循環馬達維修
1,K3/3F/I07/男廁飲水機馬達漏水維修
2,Laser Marking_步進馬達(CKM268MH-04AH-8)
3,Panasonic M41A3G2W馬達
4,(M03-57) 墊圈25*3 攪拌馬達用oring
...,...
1491,鏡面雨刷馬達(HACH PN: LZX050)
1492,鐳刻機_CT-D20馬達時規皮帶-255-5M-10
1493,變速可逆式馬達+煞車P/N:4RK25RGN-CM
1494,變速馬達 4IK25RGN-C


In [4]:
def step2(s):
    """
    filter out symbols
    :param s: string of Series
    :return: preprocessed string
    """
    for i,w in enumerate(s):
        if w != " ":
            s = s[i:]
            break
    punctuation_zh = """＂＃＄％＆＇（）＊＋－／；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏"""
    punctuation_en = string.punctuation
    re_punctuation = "[{}]+".format(punctuation_zh + punctuation_en)
    line = re.sub(re_punctuation, " ", s)
    return line
df['clean'] = df.品名.map(step2)
products_li = df.clean.tolist()
len(products_li)

1496

In [5]:
word_sentence_list = ws(products_li, segment_delimiter_set={" "}) #, segment_delimiter_set={" "}
pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
word_sentence_list

[['K1 RF', '無機', 'SCRUBBER A105', '水', '循環', '馬達維修'],
 ['K3 3F I07 ', '男廁', '飲水機', '馬達', '漏水', '維修', '  '],
 ['Laser ', 'Marking ', '步進', '馬達', ' CKM', '268', 'MH 04', 'AH 8', ' '],
 ['Panasonic ', 'M', '41', 'A3', 'G2W', '馬達'],
 [' M',
  '03 ',
  '57',
  '   ',
  '墊圈',
  '25',
  ' ',
  '3',
  '  ',
  '攪拌',
  '馬達',
  '用',
  'oring '],
 [' K', '10 BF ', '抽氣', '風車', '馬達', '修繕'],
 [' ', '維修', '  DFL', '6360', ' DD', '馬達', ' ', '含', '安裝', ' '],
 [' ', '零件', ' AOSH ', '1600', ' M', '12', '馬達', ' P', '020050104'],
 [' ', '零件', ' AOSH ', '1600', ' ', '步進', '馬達', ' P', '020050104'],
 ['08888', ' ', '1093', ' ', '000', ' ', '01', ' Z', '馬達'],
 ['08888', ' ', '1096', ' ', '000', ' ', '08', '   INDEX', '馬達'],
 ['08888', ' ', '1096', ' ', '000', ' ', '08', '  theta', '馬達'],
 ['08888', ' ', '1096', ' ', '000', ' ', '08', ' THETA', '馬達'],
 ['08888', ' ', '1096', ' ', '000', ' ', '08', ' ', '夾爪', '控制', '馬達'],
 ['0GN25', 'K ', '減速', '齒輪', ' G', '13', 'GN D AC', '小型', '標準', '馬達', ' '],
 ['0IK1GN C ', '

In [6]:
temp = []
for w_s_l, e_s_l in zip(word_sentence_list, entity_sentence_list): #take a sentence out
    t = []
    for i, x in enumerate(w_s_l): # take a tok out
        if x.isspace():
            continue
        if x[0] == ' ':
            if x[-1] == ' ':
                t.append(x[1:-1])
                continue
            t.append(x[1:])
            continue
        if x[-1] == ' ':
            t.append(x[:-1])
            continue
        t.append(x)
    temp.append(t)
word_sentence_list_ch = temp
word_sentence_list_ch

[['K1 RF', '無機', 'SCRUBBER A105', '水', '循環', '馬達維修'],
 ['K3 3F I07', '男廁', '飲水機', '馬達', '漏水', '維修'],
 ['Laser', 'Marking', '步進', '馬達', 'CKM', '268', 'MH 04', 'AH 8'],
 ['Panasonic', 'M', '41', 'A3', 'G2W', '馬達'],
 ['M', '03', '57', '墊圈', '25', '3', '攪拌', '馬達', '用', 'oring'],
 ['K', '10 BF', '抽氣', '風車', '馬達', '修繕'],
 ['維修', ' DFL', '6360', 'DD', '馬達', '含', '安裝'],
 ['零件', 'AOSH', '1600', 'M', '12', '馬達', 'P', '020050104'],
 ['零件', 'AOSH', '1600', '步進', '馬達', 'P', '020050104'],
 ['08888', '1093', '000', '01', 'Z', '馬達'],
 ['08888', '1096', '000', '08', '  INDEX', '馬達'],
 ['08888', '1096', '000', '08', ' theta', '馬達'],
 ['08888', '1096', '000', '08', 'THETA', '馬達'],
 ['08888', '1096', '000', '08', '夾爪', '控制', '馬達'],
 ['0GN25', 'K', '減速', '齒輪', 'G', '13', 'GN D AC', '小型', '標準', '馬達'],
 ['0IK1GN C', '東方', '馬達'],
 ['0IK3GN D', '東方', '馬達'],
 ['1 30', '齒輪', '減速', '馬達'],
 ['100', 'W', '馬達', 'Driver 組'],
 ['100', 'W', '馬達', '  Driver 組', '含', '安裝'],
 ['100', 'W', '馬達', 'Driver 組', '含', '安裝', '測試'

In [7]:
NER_news_li = []
for i, en_sentence in enumerate(entity_sentence_list):
    df = pd.DataFrame(columns=['Word', 'NER'])
    word = []
    for entity in en_sentence:
        word_ = entity[-1]
        if word_ in word:
            continue
        word.append(word_)
        temp = [word_, entity[-2]]
        temp = pd.Series(temp, index=df.columns)
        df = df.append(temp, ignore_index=True)
    NER_news_li.append(df)
NER_news_li[3]    

,Word,NER


In [8]:
TFIDF_df_li = []
all_corpus = []
for sentence in word_sentence_list_ch:
    all_corpus.append(" ".join(sentence))
# print(all_corpus[0])
vectoerizer = CountVectorizer(min_df=2, max_df=0.99, token_pattern='\\b\\w+\\b')
vectoerizer.fit(all_corpus)
X = vectoerizer.transform(all_corpus)
tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(X.toarray())
word = vectoerizer.get_feature_names()
weight = tfidf.toarray()
for i in range(len(weight)):
    # print("text:",i)
    df = pd.DataFrame(columns=['Word', 'Tfidf'])
    for j in range(len(word)):
        if weight[i][j] <= 0:
            continue
        temp = [word[j], weight[i][j]]
        temp = pd.Series(temp, index=df.columns)
        df = df.append(temp, ignore_index=True)
        # print(word[j],weight[i][j])
    TFIDF_df_li.append(df)
TFIDF_df_li[57]    

,Word,Tfidf
0,伺服,1.0


In [9]:
TR_df_li = []
for sentence in all_corpus:
    text_rank_words = keywords.keywords(sentence, split=True)
    all_length = len(text_rank_words)
    df = pd.DataFrame(columns=['Word', 'TR_normScore'])
    for i, words in enumerate(text_rank_words):
        word_li = words.split()
        for word in word_li:
            score = (all_length - i) / all_length
            temp = [word, score]
            temp = pd.Series(temp, index=df.columns)
            df = df.append(temp, ignore_index=True)
    all_words = df.Word.tolist()
    for i, w in enumerate(all_words):
        if len(w) < 2:
            df.iloc[i, 1] = df.iloc[i, 1] - (df.TR_normScore.mean() + 3 * df.TR_normScore.std())
    TR_df_li.append(df)
TR_df_li[3]

,Word,TR_normScore
0,m,NaN


In [10]:
# combine all
COM2_df_li = []
for ner_df, tfidf_df, tr_df in zip(NER_news_li, TFIDF_df_li, TR_df_li):
    com_df = ner_df.merge(tfidf_df, how='outer', on='Word')
    com_df = com_df.fillna(0)

    def transform(s):
        if s != 0:
            return (tfidf_df.Tfidf.median() + tr_df.TR_normScore.median())  # /2
        else:
            return 0.0

    com_df.NER = com_df.NER.map(transform)
    com_df['score'] = com_df.NER
    com2_df = com_df.merge(tr_df, on='Word', how='outer')
#     com2_df = com2_df.merge(tr_df, on='Word', how='outer')
    com2_df = com2_df.fillna(0)
    com2_df['score'] = com2_df.score + com2_df.TR_normScore + com2_df.Tfidf
    COM2_df_li.append(com2_df)
COM2_df_li[0]

,Word,NER,Tfidf,score,TR_normScore
0,馬達維修,1.391084,0.347075,1.738159,0.0
1,k1,0.000000,0.435094,0.435094,0.0
2,rf,0.000000,0.276758,1.276758,1.0
3,循環,0.000000,0.465373,0.465373,0.0
4,水,0.000000,0.327042,0.327042,0.0
5,無機,0.000000,0.538618,0.538618,0.0



* sort by score and apply rule
* keep the score and output

In [11]:
def isEnglish(s):
    return s.translate(None, string.punctuation).isalnum()
def nonKeyword_rules(s):
    """
    convert bad key word to nan
    Input: string of keyword
    """
    if len(s)<2:
#         print(1)
        return 'nan'
    if " " in s:
#         print(2)
        return 'nan'
    if not s.isalpha():
#         print(3)
        return 'nan'
    if (s.isalpha()) and (len(s)<4) and (s.isascii()):
#         print(4)
        return 'nan'
#     print(5)
    return s
 
    
len(COM2_df_li)   

1496

In [12]:
cols = ['Keyword','Score']
df_key = pd.DataFrame(columns=cols)

for com2_df in COM2_df_li:
    com2_df['keyWord'] = com2_df.Word.map(nonKeyword_rules)
    com2_df = com2_df.sort_values(['score'],ascending=False)
    keywords = com2_df.keyWord.tolist()
    scores = com2_df.score.tolist()
    need_keyword_li = []
    need_score_li = []
    for keyword, score in zip(keywords, scores):
        if keyword != 'nan':
            need_keyword_li.append(keyword)
            need_score_li.append(str(score))
#     for keyword, score in zip(need_keyword_li,need_score_li):
    keywords = "、".join(need_keyword_li)
    scores = "、".join(need_score_li)
    temp = pd.Series([keywords,scores],index=df_key.columns)
    df_key = df_key.append(temp, ignore_index=True)
    
df_key

,Keyword,Score
0,馬達維修、無機、循環,1.7381589587485766、0.5386184302599629、0.465372...
1,漏水、維修、飲水機、馬達,0.4649589923166118、0.27017107978812627、0.26551...
2,marking、laser、步進、馬達,1.3253226240444504、0.32532262404445045、0.22607...
3,panasonic、馬達,0.3792578703997766、0.07872284442006398
4,墊圈、oring、攪拌、馬達,0.41418171803803017、0.40083543739416144、0.3899...
...,...,...
1491,鏡面、hach、雨刷、馬達,0.4415699543910578、0.42395749116691417、0.35669...
1492,鐳刻機、皮帶、馬達,0.3720016784732557、0.28566367044310115、0.05588...
1493,可逆式、變速、煞車、馬達,1.4058734036394915、0.38968473336386894、0.33275...
1494,變速、馬達,0.6885185426638489、0.1077337912734661


In [13]:
final_df = df_ori.merge(df_key, left_index=True, right_index=True)
final_df = final_df.drop(['clean'],axis=1)
final_df.to_excel('../Results/R10F_products_Leo.xlsx',index=False)
final_df

,品名,Keyword,Score
0,K1_RF無機SCRUBBER_A105水循環馬達維修,馬達維修、無機、循環,1.7381589587485766、0.5386184302599629、0.465372...
1,K3/3F/I07/男廁飲水機馬達漏水維修,漏水、維修、飲水機、馬達,0.4649589923166118、0.27017107978812627、0.26551...
2,Laser Marking_步進馬達(CKM268MH-04AH-8),marking、laser、步進、馬達,1.3253226240444504、0.32532262404445045、0.22607...
3,Panasonic M41A3G2W馬達,panasonic、馬達,0.3792578703997766、0.07872284442006398
4,(M03-57) 墊圈25*3 攪拌馬達用oring,墊圈、oring、攪拌、馬達,0.41418171803803017、0.40083543739416144、0.3899...
...,...,...,...
1491,鏡面雨刷馬達(HACH PN: LZX050),鏡面、hach、雨刷、馬達,0.4415699543910578、0.42395749116691417、0.35669...
1492,鐳刻機_CT-D20馬達時規皮帶-255-5M-10,鐳刻機、皮帶、馬達,0.3720016784732557、0.28566367044310115、0.05588...
1493,變速可逆式馬達+煞車P/N:4RK25RGN-CM,可逆式、變速、煞車、馬達,1.4058734036394915、0.38968473336386894、0.33275...
1494,變速馬達 4IK25RGN-C,變速、馬達,0.6885185426638489、0.1077337912734661
